In [1]:
import json

# ### 第一部分：对于训练数据的处理：读取文件和预处理

# - ```文本的读取```： 需要从文本中读取数据，此处需要读取的文件是```dev-v2.0.json```，并把读取的文件存入一个列表里（list）
# - ```文本预处理```： 对于问题本身需要做一些停用词过滤等文本方面的处理
# - ```可视化分析```： 对于给定的样本数据，做一些可视化分析来更好地理解数据


# #### 1.1节： 文本的读取
# 把给定的文本数据读入到```qlist```和```alist```当中，这两个分别是列表，其中```qlist```是问题的列表，```alist```是对应的答案列表

In [2]:
# #### 1.1节： 文本的读取
# 把给定的文本数据读入到```qlist```和```alist```当中，这两个分别是列表，其中```qlist```是问题的列表，```alist```是对应的答案列表
def read_corpus():
    """
    读取给定的语料库，并把问题列表和答案列表分别写入到 qlist, alist 里面。 在此过程中，不用对字符换做任何的处理（这部分需要在 Part 2.3里处理）
    qlist = ["问题1"， “问题2”， “问题3” ....]
    alist = ["答案1", "答案2", "答案3" ....]
    务必要让每一个问题和答案对应起来（下标位置一致）
    """
    # 问题列表
    qlist = []
    # 答案列表
    alist = []
    # 文件名称
    filename = 'data/train-v2.0.json'
    # 加载json文件
    datas = json.load(open(filename, 'r'))
    data = datas['data']
    for d in data:
        paragraph = d['paragraphs']
        for p in paragraph:
            qas = p['qas']
            for qa in qas:
                # 处理is_impossible为True时answers空
                if (not qa['is_impossible']):
                    qlist.append(qa['question'])
                    alist.append(qa['answers'][0]['text'])
    # 如果它的条件返回错误，则终止程序执行
    # 这行代码是确保长度一样
    assert len(qlist) == len(alist)
    return qlist, alist

In [3]:
# 读取给定的语料库，并把问题列表和答案列表分别写入到 qlist, alist
qlist, alist = read_corpus()

# ### 1.2 理解数据（可视化分析/统计信息）
# 统计一下在qlist中总共出现了多少个单词？ 总共出现了多少个不同的单词(unique word)？
# 这里需要做简单的分词，对于英文我们根据空格来分词即可，其他过滤暂不考虑（只需分词）
words_qlist = dict()
for q in qlist:
    # 以空格为分词，都转为小写
    words = q.strip().split(' ')
    for w in words:
        if w.lower() in words_qlist:
            words_qlist[w.lower()] += 1
        else:
            words_qlist[w.lower()] = 1
word_total = len(words_qlist)
print("qlist的单词统计：",word_total)

qlist的单词统计： 57807


In [4]:
# #### 1.3 文本预处理
# 此部分需要做文本方面的处理。 以下是可以用到的一些方法：
#
# - 1. 停用词过滤 （去网上搜一下 "english stop words list"，会出现很多包含停用词库的网页，或者直接使用NLTK自带的）
# - 2. 转换成lower_case： 这是一个基本的操作
# - 3. 去掉一些无用的符号： 比如连续的感叹号！！！， 或者一些奇怪的单词。
# - 4. 去掉出现频率很低的词：比如出现次数少于10,20.... （想一下如何选择阈值）
# - 5. 对于数字的处理： 分词完只有有些单词可能就是数字比如44，415，把所有这些数字都看成是一个单词，这个新的单词我们可以定义为 "#number"
# - 6. lemmazation： 在这里不要使用stemming， 因为stemming的结果有可能不是valid word。

In [5]:
pip install numpy==1.16.4

  Using cached numpy-1.16.4-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl (13.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.19.0.dev0 requires numpy>=1.17, but you have numpy 1.16.4 which is incompatible.
thinc 8.0.13 requires typing-extensions<4.0.0.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.1.1 which is incompatible.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.16.4 which is incompatible.
statsmodels 0.13.2 requires numpy>=1.17, but you have numpy 1.16.4 which is incompatible.
spacy 3.2.3 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.1.1 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.16.4 which is incompatible.
scikit-image 0.19.2 requires

In [6]:
import nltk
from nltk.corpus import stopwords
import codecs
import re


# 去掉一些无用的符号
def tokenizer(ori_list):
    # 利用正则表达式去掉无用的符号
    # compile 函数用于编译正则表达式，[]用来表示一组字符
    # \s匹配任意空白字符，等价于 [\t\n\r\f]。
    SYMBOLS = re.compile('[\s;\"\",.!?\\/\[\]\{\}\(\)-]+')
    new_list = []
    for q in ori_list:
        # split 方法按照能够匹配的子串将字符串分割后返回列表
        words = SYMBOLS.split(q.lower().strip())
        new_list.append(' '.join(words))
    return new_list

/opt/anaconda3/lib/python3.7/site-packages/scipy/__init__.py:149: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.4
  UserWarning)


In [7]:
# 去掉question的停用词
def removeStopWord(ori_list):
    new_list = []
    # nltk中stopwords包含what等，但是在QA问题中，这算关键词，所以不看作关键词
    restored = ['what', 'when', 'which', 'how', 'who', 'where']
    # nltk中自带的停用词库
    english_stop_words = list(
        set(stopwords.words('english')))  # ['what','when','which','how','who','where','a','an','the']
    # 将在QA问答系统中不算停用词的词去掉
    for w in restored:
        english_stop_words.remove(w)
    for q in ori_list:
        # 将每个问句的停用词去掉
        sentence = ' '.join([w for w in q.strip().split(' ') if w not in english_stop_words])
        # 将去掉停用词的问句添加至列表中
        new_list.append(sentence)
    return new_list

In [8]:
def removeLowFrequence(ori_list, vocabulary, thres=10):
    """
    去掉低频率的词
    :param ori_list: 预处理后的问题列表
    :param vocabulary: 词频率字典
    :param thres: 频率阈值，可以基于数据实际情况进行调整
    :return: 新的问题列表
    """
    # 根据thres筛选词表，小于thres的词去掉
    new_list = []
    for q in ori_list:
        sentence = ' '.join([w for w in q.strip().split(' ') if vocabulary[w] >= thres])
        new_list.append(sentence)
    return new_list

In [9]:
def replaceDigits(ori_list, replace='#number'):
    """
    将数字统一替换为replace,默认#number
    :param ori_list: 预处理后的问题列表
    :param replace:
    :return:
    """
    # 编译正则表达式：匹配1个或多个数字
    DIGITS = re.compile('\d+')
    new_list = []
    for q in ori_list:
        # re.sub用于替换字符串中的匹配项，相当于在q中查找连续的数字替换为#number
        q = DIGITS.sub(replace, q)
        # 将处理后的问题字符串添加到新列表中
        new_list.append(q)
    return new_list

In [10]:
def createVocab(ori_list):
    """
    创建词表，统计所有单词总数与每个单词总数
    :param ori_list:预处理后的列表
    :return:所有单词总数与每个单词总数
    """
    count = 0
    vocab_count = dict()
    for q in ori_list:
        words = q.strip().split(' ')
        count += len(words)
        for w in words:
            if w in vocab_count:
                vocab_count[w] += 1
            else:
                vocab_count[w] = 1
    return vocab_count, count

In [11]:
def writeFile(oriList, filename):
    """
    将处理后的问题列表写入到文件中
    :param oriList: 预处理后的问题列表
    :param filename: 文件名
    """
    with codecs.open(filename, 'w', 'utf8') as Fout:
        for q in oriList:
            Fout.write(q + u'\n')

In [12]:
def writeVocab(vocabulary, filename):
    """
    将词表写入到文件中
    :param vocabulary: 词表
    :param filename: 文件名
    """
    sortedList = sorted(vocabulary.items(), key=lambda d: d[1])
    with codecs.open(filename, 'w', 'utf8') as Fout:
        for (w, c) in sortedList:
            Fout.write(w + u':' + str(c) + u'\n')

In [13]:
# 去掉一些无用的符号
new_list = tokenizer(qlist)
# 停用词过滤
new_list = removeStopWord(new_list)
# 数字处理-将数字替换为#number
new_list = replaceDigits(new_list)
# 创建词表并统计所有单词数目
vocabulary, count = createVocab(new_list)
# 去掉低频率的词
new_list = removeLowFrequence(new_list, vocabulary, 5)
# 重新统计词频
vocab_count, count = createVocab(new_list)
# 将词表写入到文件“train.vocab”中
writeVocab(vocab_count, "train.vocab")
qlist = new_list
# print(qlist[:100])

In [14]:
# ### 第二部分： 文本的表示
# 文本处理完之后，我们需要做文本表示，这里有3种方式：
#
# - 1. 使用```tf-idf vector```
# - 2. 使用embedding技术如```word2vec```, ```bert embedding```等

# #### 2.1 使用tf-idf表示向量
# 把```qlist```中的每一个问题的字符串转换成```tf-idf```向量, 转换之后的结果存储在```X```矩阵里。
# # ``X``的大小是： ``N* D``的矩阵。 这里``N``是问题的个数（样本个数），``D``是词典库的大小


In [15]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np


def computeTF(vocab, c):
    """
    计算每个词的词频
    :param vocab: 词频字典:键是单词，值是所有问句中单词出现的次数
    :param c: 单词总数
    :return: TF
    """
    # 初始化TF
    TF = np.ones(len(vocab))
    # 词频字典
    word2id = dict()
    # 单词字典
    id2word = dict()
    for word, fre in vocab.items():
        # 计算TF值：每个单词出现的个数/总的单词个数
        TF[len(word2id)] = 1.0 * fre / c
        id2word[len(word2id)] = word
        word2id[word] = len(word2id)
    return TF, word2id, id2word

In [17]:
def computeIDF(word2id, qlist):
    """
    计算IDF：log[问句总数/(包含单词t的问句总数+1)]
    :param word2id:单词字典
    :param qlist:问句列表
    :return:
    """
    IDF = np.ones(len(word2id))
    for q in qlist:
        # 去重
        words = set(q.strip().split())
        for w in words:
            # 统计单词出现在问句中的总数
            IDF[word2id[w]] += 1
    # 计算IDF
    IDF /= len(qlist)
    IDF = -1.0 * np.log2(IDF)
    return IDF

In [18]:
def computeSentenceEach(sentence, tfidf, word2id):
    """
    给定句子，计算句子TF-IDF,tfidf是一个1*M的矩阵,M为词表大小
    :param sentence:句子
    :param tfidf:TF-IDF向量
    :param word2id:词表
    :return:
    """
    sentence_tfidf = np.zeros(len(word2id))
    # 将问句以空格进行分割
    for w in sentence.strip().split(' '):
        if w not in word2id:
            continue
        # 碰到在词表word2id中的单词
        sentence_tfidf[word2id[w]] = tfidf[word2id[w]]
    return sentence_tfidf

In [19]:
def computeSentence(qlist, word2id, tfidf):
    """
    把```qlist```中的每一个问题的字符串转换成```tf-idf```向量, 转换之后的结果存储在```X```矩阵里
    :param qlist: 问题列表
    :param word2id: 词表(字典形式)
    :param tfidf: TF-IDF(与词表中的键一一对应)
    :return: X矩阵
    """
    # 对所有句子分别求tfidf
    X_tfidf = np.zeros((len(qlist), len(word2id)))
    for i, q in enumerate(qlist):
        X_tfidf[i] = computeSentenceEach(q, tfidf, word2id)
        # print(X_tfidf[i])
    return X_tfidf

In [20]:
# 计算每个词的TF,词表字典
TF, word2id, id2word = computeTF(vocab_count, count)
# 计算IDF：log[问句总数/(包含单词t的问句总数+1)]
IDF = computeIDF(word2id, qlist)
# 用TF，IDF计算最终的tf-idf,定义一个tf-idf的vectorizer
vectorizer = np.multiply(TF, IDF)
# 把```qlist```中的每一个问题的字符串转换成```tf-idf```向量, 转换之后的结果存储在```X```矩阵里
X_tfidf = computeSentence(qlist, word2id, vectorizer)

In [21]:
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 2.4 MB/s eta 0:00:011
  ERROR: Command errored out with exit status 1:
   command: /opt/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/5m/hbh9nnpn5y92cglh1hvnxmc40000gn/T/pip-install-637_iqt_/python-levenshtein_4ef2686472114fffa8fcd5e246e2546d/setup.py'"'"'; __file__='"'"'/private/var/folders/5m/hbh9nnpn5y92cglh1hvnxmc40000gn/T/pip-install-637_iqt_/python-levenshtein_4ef2686472114fffa8fcd5e246e2546d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/5m/hbh9nnpn5y92cglh1hvnxmc40000gn/T/pip-wheel-levrh52h
       cwd: /private/var/folders/5m/hbh9nnpn5y92cglh1hvnxmc40000gn/T/pip-install-637_iqt_/python-levenshtein_4ef2686472114ff

Failed to build python-Levenshtein
    Running setup.py install for python-Levenshtein ... error
    ERROR: Command errored out with exit status 1:
     command: /opt/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/5m/hbh9nnpn5y92cglh1hvnxmc40000gn/T/pip-install-637_iqt_/python-levenshtein_4ef2686472114fffa8fcd5e246e2546d/setup.py'"'"'; __file__='"'"'/private/var/folders/5m/hbh9nnpn5y92cglh1hvnxmc40000gn/T/pip-install-637_iqt_/python-levenshtein_4ef2686472114fffa8fcd5e246e2546d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/5m/hbh9nnpn5y92cglh1hvnxmc40000gn/T/pip-record-67agxqkn/install-record.txt --single-version-externally-managed --compile --install-headers /opt/

In [22]:
# #### 2.2 使用wordvec + average pooling
# 词向量方面需要下载： https://nlp.stanford.edu/projects/glove/ （请下载``glove.6B.zip``），并使用``d=200``的词向量（200维）。
# 国外网址如果很慢，可以在百度上搜索国内服务器上的。 每个词向量获取完之后，即可以得到一个句子的向量。 我们通过``average pooling``来实现句子的向量。
# 基于Glove向量获取句子向量
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

AttributeError: module 'numpy.random' has no attribute 'default_rng'

In [ ]:
def loadEmbedding(filename):
    """
    加载glove模型，转化为word2vec，再加载到word2vec模型
    这两种模型形式上是一样的，在数据的保存形式上有略微的差异
    :param filename: glove文件
    :return: word2vec模型
    """
    word2vec_temp_file = 'word2vec_temp.txt'
    # 加载glove模型，转化为word2vec
    glove2word2vec(filename, word2vec_temp_file)
    # 再加载到word2vec模型
    model = KeyedVectors.load_word2vec_format(word2vec_temp_file)
    return model

In [ ]:
def computeGloveSentenceEach(sentence, embedding):
    """
    :param sentence:问题
    :param embedding:wordvec模型
    :return:
    """
    # 查找句子中每个词的embedding,将所有embedding进行加和求均值
    emb = np.zeros(200)
    words = sentence.strip().split(' ')
    for w in words:
        # 如果单词是新词，则重命名为'unknown'
        if w not in embedding:
            # 没有lookup的即为unknown
            w = 'unknown'
        # 将所有embedding进行加和求均值
        emb += embedding.get_vector(w)
        # emb += embedding[w]
    return emb / len(words)

In [ ]:
def computeGloveSentence(qlist, embedding):
    """
    对每一个句子来构建句子向量
    :param qlist:问题列表
    :param embedding:word2vec模型
    :return:
    """
    # 对每一个句子进行求均值的embedding
    X_w2v = np.zeros((len(qlist), 200))
    for i, q in enumerate(qlist):
        # 编码每一个问题
        X_w2v[i] = computeGloveSentenceEach(q, embedding)
        # print(X_w2v)
    return X_w2v

In [ ]:
# 加载到word2vec模型
# 这是 D*H的矩阵，这里的D是词典库的大小， H是词向量的大小。 这里面我们给定的每个单词的词向量，
emb = loadEmbedding('data/glove.6B.200d.txt')
# 初始化完emb之后就可以对每一个句子来构建句子向量了，这个过程使用average pooling来实现
X_w2v = computeGloveSentence(qlist, emb)
print(X_w2v)

In [ ]:
!pip install bert_embedding

In [ ]:
# TODO 基于BERT的句子向量计算
from bert_embedding import BertEmbedding

sentence_embedding = np.ones((len(qlist), 768))
# 加载Bert模型，model，dataset_name,须指定
bert_embedding = BertEmbedding(model='bert_12_768_12', dataset_name='wiki_multilingual_cased')
# 查询所有句子的Bert  embedding
all_embedding = bert_embedding(qlist, 'sum')
for q in qlist:
    for i in range(len(all_embedding)):
        print(all_embedding[i][1])
        # 将每一个句子中每一个词的向量进行拼接求平均得到句子向量
        sentence_embedding[i] = np.sum(all_embedding[i][1], axis=0) / len(q.strip().split(' '))
        if i == 0:
            print(sentence_embedding[i])

In [ ]:
# 每一个句子的向量结果存放在X_bert矩阵里。行数为句子的总个数，列数为一个句子embedding大小。
X_bert = sentence_embedding

In [ ]:
# ### 第三部分： 相似度匹配以及搜索
# 在这部分里，我们需要把用户每一个输入跟知识库里的每一个问题做一个相似度计算，从而得出最相似的问题。但对于这个问题，时间复杂度其实很高，所以我们需要结合倒排表来获取相似度最高的问题，从而获得答案。

# In[ ]:


# #### 3.1 tf-idf + 余弦相似度
# 我们可以直接基于计算出来的``tf-idf``向量，计算用户最新问题与库中存储的问题之间的相似度，从而选择相似度最高的问题的答案。
# 这个方法的复杂度为``O(N)``， ``N``是库中问题的个数。

import queue as Q

# 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
que = Q.PriorityQueue()

In [ ]:
def cosineSimilarity(vec1, vec2):
    # 定义余弦相似度,余弦相似度越大，两个向量越相似
    return np.dot(vec1, vec2.T) / (np.sqrt(np.sum(vec1 ** 2)) * np.sqrt(np.sum(vec2 ** 2)))


def get_top_results_tfidf_noindex(query):
    """
    给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    :param query:用户新问题
    :return:
    """
    """
    给定用户输入的问题 query, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 对于用户的输入 query 首先做一系列的预处理(上面提到的方法)，然后再转换成tf-idf向量（利用上面的vectorizer)
    2. 计算跟每个库里的问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    top = 5
    # 将用户输入的新问题用tf-idf来表示
    query_tfidf = computeSentenceEach(query.lower(), vectorizer, word2id)
    for i, vec in enumerate(X_tfidf):
        # 计算原问题与用户输入的新问题的相似度
        result = cosineSimilarity(vec, query_tfidf)
        # print(result)
        # 存放到大顶堆里面
        que.put((-1 * result, i))
    i = 0
    top_idxs = []
    while (i < top and not que.empty()):
        # top_idxs存放相似度最高的（存在qlist里的）问题的下标
        top_idxs.append(que.get()[1])
        i += 1
    print(top_idxs)
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]

In [ ]:
# 给定用户输入的问题 query, 返回最有可能的TOP 5问题的答案。
results = get_top_results_tfidf_noindex('In what city and state did Beyonce  grow up')
print(results)

In [ ]:
word_doc = dict()
# key:word,value:包含该词的句子序号的列表
for i, q in enumerate(qlist):
    words = q.strip().split(' ')
    for w in set(words):
        if w not in word_doc:
            # 没在word_doc中的，建立一个空listi
            word_doc[w] = set([])
        word_doc[w] = word_doc[w] | set([i])

# 定一个一个简单的倒排表，是一个map结构。 循环所有qlist一遍就可以
inverted_idx = word_doc

In [ ]:
# #### 3.3 语义相似度
# 读取语义相关的单词
import codecs


def get_related_words(filename):
    """
    从预处理的相似词的文件加载相似词信息
    文件格式w1 w2 w3..w11,其中w1为原词，w2-w11为w1的相似词
    :param filename: 文件名
    :return:
    """
    related_words = {}
    with codecs.open(filename, 'r', 'utf8') as Fin:
        lines = Fin.readlines()
    for line in lines:
        words = line.strip().split(' ')
        # 键为原词，值为相似词
        related_words[words[0]] = words[1:]
    return related_words

In [ ]:
# 从预处理的相似词的文件加载相似词信息
related_words = get_related_words('data/related_words.txt')

In [ ]:
# #### 3.4 利用倒排表搜索
# 在这里，我们使用倒排表先获得一批候选问题，然后再通过余弦相似度做精准匹配，这样一来可以节省大量的时间。搜索过程分成两步：
# - 使用倒排表把候选问题全部提取出来。首先，对输入的新问题做分词等必要的预处理工作，然后对于句子里的每一个单词，从``related_words``里提取出跟它意思相近的top 10单词， 然后根据这些top词从倒排表里提取相关的文档，把所有的文档返回。 这部分可以放在下面的函数当中，也可以放在外部。
# - 然后针对于这些文档做余弦相似度的计算，最后排序并选出最好的答案。

In [ ]:
import queue as Q


def cosineSimilarity(vec1, vec2):
    # 定义余弦相似度
    return np.dot(vec1, vec2.T) / (np.sqrt(np.sum(vec1 ** 2)) * np.sqrt(np.sum(vec2 ** 2)))


def getCandidate(query):
    """
    根据查询句子中每个词及其10个相似词所在的序号列表，求交集
    :param query: 问题
    :return:
    """
    searched = set()
    for w in query.strip().split(' '):
        # 如果单词不在word2id中或者不在倒排表中
        if w not in word2id or w not in inverted_idx:
            continue
        # 搜索原词所在的序号列表
        if len(searched) == 0:
            searched = set(inverted_idx[w])
        else:
            searched = searched & set(inverted_idx[w])
        # 搜索相似词所在的列表
        if w in related_words:
            for similar in related_words[w]:
                searched = searched & set(inverted_idx[similar])
    return searched

In [ ]:
def get_top_results_tfidf(query):
    """
    基于TF-IDF,给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate （需要使用related_words).
    2. 对于候选文档，计算跟输入问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    top = 5
    # 计算给定句子的TF-IDF
    query_tfidf = computeSentenceEach(query, vectorizer, word2id)
    # 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
    results = Q.PriorityQueue()
    # 利用倒排索引表获取相似问题
    searched = getCandidate(query)
    # print(len(searched))
    for candidate in searched:
        # 计算candidate与query的余弦相似度
        result = cosineSimilarity(query_tfidf, X_tfidf[candidate])
        # 优先级队列中保存相似度和对应的candidate序号
        # -1保证降序
        results.put((-1 * result, candidate))
    i = 0
    # top_idxs存放相似度最高的（存在qlist里的）问题的索引
    top_idxs = []
    # hint: 利用priority queue来找出top results.
    while i < top and not results.empty():
        top_idxs.append(results.get()[1])
        i += 1
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]

In [ ]:
def get_top_results_w2v(query):
    """
    基于word2vec，给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate （需要使用related_words).
    2. 对于候选文档，计算跟输入问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    # embedding用glove
    top = 5
    # 利用glove将问题进行编码
    query_emb = computeGloveSentenceEach(query, emb)
    # 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
    results = Q.PriorityQueue()
    # 利用倒排索引表获取相似问题
    searched = getCandidate(query)
    for candidate in searched:
        # 计算candidate与query的余弦相似度
        result = cosineSimilarity(query_emb, X_w2v[candidate])
        # 优先级队列中保存相似度和对应的candidate序号
        # -1保证降序
        results.put((-1 * result, candidate))
    # top_idxs存放相似度最高的（存在qlist里的）问题的索引
    top_idxs = []
    i = 0
    # hint: 利用priority queue来找出top results.
    while i < top and not results.empty():
        top_idxs.append(results.get()[1])
        i += 1
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]

In [ ]:
def get_top_results_bert(query):
    """
    给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate （需要使用related_words).
    2. 对于候选文档，计算跟输入问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    top = 5
    # embedding用Bert embedding
    query_emb = np.sum(bert_embedding([query], 'sum')[0][1], axis=0) / len(query.strip().split())
    # 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
    results = Q.PriorityQueue()
    # 利用倒排索引表获取相似问题
    searched = getCandidate(query)
    for candidate in searched:
        # 计算candidate与query的余弦相似度
        result = cosineSimilarity(query_emb, X_bert[candidate])
        # 优先级队列中保存相似度和对应的candidate序号
        # -1保证降序
        results.put((-1 * result, candidate))
    # top_idxs存放相似度最高的（存在qlist里的）问题的索引
    top_idxs = []
    i = 0
    # hint: 利用priority queue来找出top results.
    while i < top and not results.empty():
        top_idxs.append(results.get()[1])
        i += 1
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]
